In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Getting data
train_data = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv")
test_data = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv")
sample_solution = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv")

In [ ]:
# Drop unnecessary data
train_data = train_data.drop(['flight', 'source_city', 'destination_city', 'departure_time', 'arrival_time'], axis=1)
test_data = test_data.drop(['flight', 'source_city', 'destination_city', 'departure_time', 'arrival_time'], axis=1)
# Display the updated DataFrame
train_data.head()


In [ ]:
# Choose categorical data, in order to encode
categorical_cols = ['airline', 'stops', 'class']

# Use OneHotEncoder for categorical data
encoder = OneHotEncoder(sparse_output=False)  # Dense output is already provided
encoded_categorical = pd.DataFrame(encoder.fit_transform(train_data[categorical_cols]))
encoded_categorical.columns = encoder.get_feature_names_out(categorical_cols)

# Drop original categorical columns and concatenate encoded columns
train_data = train_data.drop(categorical_cols, axis=1)
train_data = pd.concat([train_data, encoded_categorical], axis=1)

# Repeat the same for test data
test_encoded_categorical = pd.DataFrame(encoder.transform(test_data[categorical_cols]))
test_encoded_categorical.columns = encoder.get_feature_names_out(categorical_cols)
test_data = test_data.drop(categorical_cols, axis=1)
test_data = pd.concat([test_data, test_encoded_categorical], axis=1)

# Scale numerical features using StandarScaler()
scaler = StandardScaler()
numerical_cols = ['duration', 'days_left']
train_data[numerical_cols] = scaler.fit_transform(train_data[numerical_cols])
test_data[numerical_cols] = scaler.transform(test_data[numerical_cols])
train_data.corrwith(train_data['price']).abs().sort_values(ascending=False)

In [ ]:
# Prepare train data before splitting
X = train_data.drop(['id','price'], axis=1)
y = train_data['price']

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# Train a Random Forest Regressor
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
# Make predictions on the validation set
y_pred = model.predict(X_val)

# Check its errors
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

In [ ]:
# Plotting predicted values and real values
plt.scatter(y_pred, y_val)
plt.title("Plot")
plt.xlabel("Predicted Values")
plt.ylabel("Real Values")
plt.show()

In [ ]:
# Predict on test data
X_test = test_data.drop('id', axis=1)  
test_predictions = model.predict(X_test) 

# Add predictions to the test dataset
test_data['price'] = test_predictions  

# Prepare the solution file with only 'id' and 'price' columns
solution_data = test_data[['id', 'price']]
print(solution_data.head(10))  
# Save solution
solution_data.to_csv('solution_data.csv',index=False)
